In [1]:
import kagglehub
import pandas as pd
import hopsworks
import os

path = kagglehub.dataset_download("aglitoiumarius/rsolana-comments-202001202204")

print("Path to dataset files:", path)

# Try reading with the Python engine to avoid buffer overflow errors
try:
    df = pd.read_csv(f"{path}/solana_2020__2022_04.csv", engine="python")
except Exception as e:
    print("Error reading CSV with python engine:", e)


c:\Users\samyz\miniconda\envs\ca\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\samyz\.cache\kagglehub\datasets\aglitoiumarius\rsolana-comments-202001202204\versions\1


In [2]:
df.head()

,created_utc,subreddit,body,score
0,1584529482,solana,SOL SUN SUN LIGHTThis is the light kin nee...,1
1,1584496391,solana,Tonight is just pre bidding The actual auct...,1
2,1584280821,solana,Yo yo yo u rhlm39 ma big dawwwg how you been b...,1
3,1584272064,solana,Solana is defiantly a top notch project,1
4,1584261728,solana,This is Great The real scaling in Crypto is...,1


# Processing

There is no much processing since there are no redundant features apart from score. We want to create our own score to not have drift with the inference pipeline.

In [18]:
df_sol = df.drop(columns=['score'])
df_sol.dropna(subset=['body'], inplace=True)

df_sol.head()

,created_utc,subreddit,body
0,1584529482,solana,SOL SUN SUN LIGHTThis is the light kin nee...
1,1584496391,solana,Tonight is just pre bidding The actual auct...
2,1584280821,solana,Yo yo yo u rhlm39 ma big dawwwg how you been b...
3,1584272064,solana,Solana is defiantly a top notch project
4,1584261728,solana,This is Great The real scaling in Crypto is...


In [19]:
df_sol.isna().sum()

created_utc    0
subreddit      0
body           0
dtype: int64

There are a lot of comments. That is great. However, it stops at march 2022

In [20]:
project = hopsworks.login(api_key_value=os.getenv("HOPSWORKS_API_KEY"))

fs = project.get_feature_store()

reddit_historical_comments_fg = fs.get_or_create_feature_group(
    name="reddit_historical_comments",
    version=1,
    description="Raw Reddit comments for SOL sentiment analysis",
    primary_key=['created_utc'],
    event_time='created_utc',
    online_enabled=False
)

reddit_historical_comments_fg.insert(df_sol)
print(f"✅ {len(df_sol)} comments uploaded to Hopsworks")

2026-01-01 18:12:31,612 INFO: Closing external client and cleaning up certificates.
2026-01-01 18:12:31,619 INFO: Connection closed.
2026-01-01 18:12:31,624 INFO: Initializing external client
2026-01-01 18:12:31,625 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2026-01-01 18:12:33,666 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279131








Uploading Dataframe: 100.00% |██████████| Rows 239693/239693 | Elapsed Time: 00:18 | Remaining Time: 00:00


Launching job: reddit_historical_comments_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279131/jobs/named/reddit_historical_comments_1_offline_fg_materialization/executions
✅ 239693 comments uploaded to Hopsworks
